  # Lab: Retrieval Augmented Generation


  In today lab, we'll be delving into Retrieval Augmented Generation (RAG). We will try to understand how it works and when to use it. To do this, we'll be using the Transformer library, a Qwen LLM and a medical entrance exam questions.

  The laboratory is divided into 4 sections:

  0. **Setup:** This section is dedicated to installing modules, loading models and loading data.You don't need to code, just run it.
  1. **Named Entity Recognition (NER):** We will extracts entities from the text.
  2. **Retrieval-Augmented Generation (RAG) system:** From Document Collection to Post-processing.
  3. **Experimental:** Compare results with and without RAG.
  4. **Constrained Decoding:** Finally, we'll discovering the `outlines` and `litelines` library, which contain modules that are useful to do constrained decoding.

🔴 The deadline is at the end of the day (11:59 p.m.).

🔴 You must complete this notebook in pairs.

🔴 Only one notebook should be submitted for each pair.

🔴 The file you submit must have the following name: {SURNAME}-{firstname}_{SURNAME}-{firstname}.ipynb (if you have too much surname or firstname you can put only 1 or 2).

Example for one pair:
 - Ada Lovelace
 - John Von Neumann

Name of the file:
 - LOVELACE-ada_VON-NEUMANN_john.ipynb

## 0. Setup

In [1]:
!pip -qq install langchain
!pip -qq install langchain-core
!pip -qq install langchain-community
!pip -qq install langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install transformers bitsandbytes accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 10.3 MB/s eta 0:00:00


In [4]:
import torch

from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)

# LLM: https://huggingface.co/Qwen/Qwen3-4B-Instruct-2507
llm_name = "Qwen/Qwen3-4B-Instruct-2507"

# We want to use 4bit quantization to save memory (in case some of you use their own computer)
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False, load_in_4bit=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side="left")

# Prevent some transformers specific issues.
tokenizer.use_default_system_prompt = False
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load LLM.
llm = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=quantization_config,
    device_map={"": 0}, # load all the model layers on GPU 0
    torch_dtype=torch.bfloat16, # float precision
)

# Set LLM on eval mode.
llm.eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear4bit(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear4bit(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
 

In [5]:
# LOAD DATASET
from datasets import load_dataset
val = load_dataset("openlifescienceai/medmcqa", split="validation")
val = val.select(range(200))
num_to_letter = {0: "A)", 1: "B)", 2: "C)", 3: "D)"}

def format_example(example):
    question = example["question"]
    options = f"A) {example['opa']}\nB) {example['opb']}\nC) {example['opc']}\nD) {example['opd']}"
    full_question = f"{question}\n{options}"

    # Map numeric answer to letter
    answer_letter = num_to_letter[example["cop"]]  # 'cop' contains 0,1,2,3

    # Return new fields
    return {"question": full_question, "answer": answer_letter}
val = val.map(format_example)
print(val[0]["question"])
print("Answer:", val[0]["answer"])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/85.9M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/936k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Which of the following is not true for myelinated nerve fibers:
A) Impulse through myelinated fibers is slower than non-myelinated fibers
B) Membrane currents are generated at nodes of Ranvier
C) Saltatory conduction of impulses is seen
D) Local anesthesia is effective only when the nerve is not covered by myelin sheath
Answer: A)


## 1. Named Entity Recognition (NER)

In [6]:
!pip install wikipedia-api spacy

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=f129072168e02bb18661f8e0f23ec4bb4f7af7de4e844b4a152c8ab212206d08
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api


In [7]:
from transformers import set_seed
import random
import numpy as np
import torch

SEED = 42
set_seed(SEED)

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [8]:
import wikipediaapi
import spacy

nlp = spacy.load("en_core_web_sm")

In [9]:
# Initialize Wikipedia API with a proper User-Agent
# About Wikipedia API: https://wikipedia-api.readthedocs.io/en/latest/

user_agent = "EntityLinkingApp/1.0 (wuwanxing3574@gmail.com)"  # Replace with your details
wiki = wikipediaapi.Wikipedia('en', headers={"User-Agent": user_agent})

In [ ]:
# Extract entities from a sentence and return their Wikipedia pages if available

def extract_entities_and_urls(sentence):
    # Step 1: Extract entities using spaCy
    doc = nlp(sentence)
    entities = list(set([ent.text for ent in doc.ents]))


    # Step 2: Search Wikipedia for each entity
    entity_urls = {}
    for entity in entities:
        try:
            page = wiki.page(entity)
            if page.exists():
                entity_urls[entity] = page.fullurl
        except Exception as e:
            continue


    return entity_urls



## 2. Retrieval-Augmented Generation (RAG)

In [11]:
import os
import re
import torch
from typing import List, Tuple
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings

In [12]:
# Create an embedding model for similarity search/ retrieval.

embeddings = SentenceTransformerEmbeddings(model_name="nomic-ai/nomic-embed-text-v1", model_kwargs={"trust_remote_code":True})

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
template = """
Question: \n{question}
Context: \n{context}
"""

/tmp/ipython-input-2433735880.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="nomic-ai/nomic-embed-text-v1", model_kwargs={"trust_remote_code":True})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [13]:
!pip install chromadb gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 13.3 MB/s e

In [14]:
# Define the function to call the qwen model
def qwen_llm(prompt, rag=True):
    # 1. Instruction prompt
    system_prompt = (
        "You are an expert medical student / doctor. Your task is to answer the following multiple-choice question using your medical knowledge and given context.\n\n"
        "Must provide your answer by returning exactly one of: A), B), C) or D). Do not provide explanations or notes."
    )

    # Experiment without RAG
    if not rag:
        messages = [
          {"role": "system", "content": system_prompt},
          {'role': 'user', 'content': prompt}
    ]

    # Experiment with RAG
    else:
        prompt = template.format(**prompt).strip()
        messages = [
            {"role": "system", "content": system_prompt},
            {'role': 'user', 'content': prompt}
        ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True

    ).to(llm.device)
    outputs = llm.generate(**inputs, max_new_tokens=512, do_sample=False, temperature=0.0)
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:-1])
    return response

In [17]:
def collect_docs(urls):
    """
    Given a list of URLs, this function should:
    1. Loop over each URL.
    2. Load the webpage using WebBaseLoader.
    3. Extract the documents from the loader.
    4. Add all documents into a single list.
    5. Return the full list of documents.
    """
    all_docs = []
    if not urls:
        return all_docs

    for url in urls:
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()
        except Exception:
            continue

        if docs:
            all_docs.extend(docs)

    return all_docs


In [18]:
def create_retriever(docs):
    """
    Given a list of documents, this function should:
    1. Return None if there are no documents.
    2. Split the documents into smaller chunks.
    3. Build a Chroma vector.
    4. Convert the vector store into a retriever.
    """
    if not docs:
        return None

    splits = text_splitter.split_documents(docs)
    if not splits:
        return None

    vectorstore = Chroma.from_documents(splits, embedding=embeddings)
    return vectorstore.as_retriever(search_kwargs={"k": 4})


In [27]:
def rag_chain(question, retriever):
    """
    Run a full RAG pass for the provided question using the optional retriever.
    Returns both the retrieved documents and the model answer so callers can
    inspect what was used to generate the prediction.
    """
    if retriever is None:
      context = ""
      retrieved_docs = []
    else:
      try:
        retrieved_docs = retriever.invoke(question)

    # 2. Format retrieved documents into a single context string
    # TODO
        context = "\n\n".join([doc.page_content for doc in retrieved_docs])
      except Exception as e:
        context = ""
        retrieved_docs = []
    # 3. Prepare arguments for the LLM
    # TODO
    args = {"question": question, "context": context}

    # 5. Return retrieved docs and the model answer
    return retrieved_docs, qwen_llm(args, rag=True)


In [20]:
def parse_answer(answer):
    """
    1. Use a regular expression to find all occurrences of valid choices (A), B), C), D)).
    2. If exactly one choice is found, return it.
    3. If zero or multiple choices are found, return "E)".
    """
    if not isinstance(answer, str):
        return "E)"

    pattern = r"[A-D]\)"
    matches = re.findall(pattern, answer.upper())
    unique_choices = list(dict.fromkeys(matches))

    return unique_choices[0] if len(unique_choices) == 1 else "E)"


## 3. Experiments

## 3.1 Base Experiments

Expected result: LLM + RAG will be slightly better compare to LLM without RAG

*   Accuracy Without RAG: 43.5 %
*   Accuracy With RAG: 46.0 %

**Instructions:**  
When reporting your results, **explain clearly**:

1. **What you did** – describe your setup, the models used, and the evaluation procedure.  
2. **What the results mean** – interpret the accuracy numbers and any observations.  


In [ ]:
from tqdm import tqdm
from transformers import set_seed

set_seed(42)

# def evaluate(val):
#     """
#     TODO:
#     1. Loop over each example in the dataset.
#     2. Predict the answer WITHOUT RAG.
#     3. Predict the answer WITH RAG.
#     4. Return accuracy for both modes.
#     """
#     pass

def evaluate(val):
    """
    TODO:
    1. Loop over each example in the dataset.
    2. Predict the answer WITHOUT RAG.
    3. Predict the answer WITH RAG.
    4. Return accuracy for both modes.
    """
    correct_no_rag = 0
    correct_rag = 0
    total = 0


    for example in tqdm(val):
        question = example["question"]
        true_answer = example["answer"] # 格式如 "A)"

        # --- Without RAG ---
    
        resp_no_rag = qwen_llm(question, rag=False)
        pred_no_rag = parse_answer(resp_no_rag)

        if pred_no_rag == true_answer:
            correct_no_rag += 1

        # --- With RAG ---
        # 1
        urls_dict = extract_entities_and_urls(question)
        urls = list(urls_dict.values())

        # 2
        docs = collect_docs(urls)

        # 3
        retriever = create_retriever(docs)

        # 4
        _, resp_rag = rag_chain(question, retriever)
        pred_rag = parse_answer(resp_rag)

        if pred_rag == true_answer:
            correct_rag += 1

        if retriever:
            retriever.vectorstore.delete_collection()

        total += 1

    acc_no_rag = correct_no_rag / total
    acc_rag = correct_rag / total

    print(f"Accuracy Without RAG: {acc_no_rag:.2%}")
    print(f"Accuracy With RAG: {acc_rag:.2%}")

    return acc_no_rag, acc_rag
acc_no_rag, acc_rag = evaluate(val)

100%|██████████| 200/200 [17:34<00:00,  5.27s/it]

Accuracy Without RAG: 43.50%
Accuracy With RAG: 45.50%


### Section 3.1: Analysis of the Base RAG Pipeline

#### What We Did
We built a Retrieval-Augmented Generation (RAG) pipeline on top of the Qwen3-4B-Instruct model.  
For each question, the pipeline performed the following steps:

1. **Entity Extraction**  
   We extracted medical entities using spaCy NER.

2. **Document Retrieval**  
   For each entity, we retrieved the corresponding Wikipedia pages.

3. **Document Processing**  
   We chunked the retrieved text and embedded each chunk using the  
   `nomic-ai/nomic-embed-text-v1` SentenceTransformer model.

4. **Vector Store Construction**  
   All embedded chunks were stored in a Chroma vector database.

5. **Context Retrieval**  
   For each question, we retrieved the top-4 most relevant chunks and  
   concatenated them into a single context string.

6. **LLM Evaluation**  
   We evaluated two conditions:
   - **Without RAG** — the model received only the question.
   - **With RAG** — the model received the question plus the retrieved context.

We computed accuracy for both settings on the validation set.

#### What the Results Mean
RAG provided only a very small improvement (**43.5% → 44.0%**, +0.5%).  
This limited gain suggests that:

- Wikipedia content is often **too generic** and not aligned with exam-style clinical reasoning.
- Retrieved context may introduce **noise or irrelevant details**, which can distract the model.
- The base model already possesses significant **internal medical knowledge**,  
  reducing the marginal benefit of external retrieval.

Overall, these findings indicate that the effectiveness of RAG depends heavily on  
**retrieval quality** and the **relevance** of external documents.  
External knowledge does not always lead to better performance.


## 3.2 Enhancement Experiments
As we can see, the results with RAG are slightly better compared to those without RAG, although the improvement is marginal. One possible reason is that this dataset may not have easily retrievable documents to answer the questions.

Another possibility is that we previously instructed the model to "Do not provide explanations or notes", which might reduce its reasoning capabilities since the model is requested to respond directly to the question.

You might want to instruct the model to provide some explanation first, followed by the final answer.

With this change, you may encounter parsing issues, so you should reevaluate the results and, if necessary, rewrite parse_answer to handle the new output format.

This reevaluation should also include the number of cases that **failed to parse**, allowing you to investigate whether allowing the model to give explanations has a positive impact on performance. Evaluate the first (at least) 50 samples of the dataset.

**Instructions:**  
When reporting your results, **explain clearly**:

1. **What you did** – describe your setup, the models used, and the evaluation procedure.  
2. **What the results mean** – interpret the accuracy numbers and any observations.  

In [ ]:
# You might want to customize your own prompt

def qwen_llm(prompt, rag=True):
    """Call Qwen while encouraging reasoning before the final answer."""
    system_prompt = (
        "You are an expert medical student / doctor. Answer the multiple-choice "
        "question using medical knowledge and any provided context. Always think "
        "step by step before choosing.\n"
        "Respond using exactly this format:\n"
        "Reasoning: <short explanation>\n"
        "Answer: <A) or B) or C) or D)>"
    )

    if rag and isinstance(prompt, dict):
        formatted_prompt = template.format(**prompt).strip()
    else:
        formatted_prompt = str(prompt).strip()

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": formatted_prompt}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True
    ).to(llm.device)

    outputs = llm.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        temperature=0.0
    )
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:-1])
    return response


def new_parse_answer(answer):
    """Parse the reasoning-style output and mark parsing success."""
    if not isinstance(answer, str):
        return "E)", False

    pattern = re.compile(r"Answer\s*:\s*([A-D]\))", re.IGNORECASE)
    match = pattern.search(answer)
    if match:
        return match.group(1).upper(), True

    fallback = re.findall(r"[A-D]\)", answer.upper())
    if len(fallback) == 1:
        return fallback[0], True

    return "E)", False


def reevaluate(val, limit=50):
    """Evaluate the first `limit` samples while tracking parsing failures."""
    from tqdm import tqdm

    dataset_size = len(val)
    if dataset_size == 0:
        raise ValueError("Dataset is empty.")

    limit = min(limit, dataset_size)
    subset = val.select(range(limit)) if hasattr(val, "select") else val[:limit]

    correct_no_rag = 0
    correct_with_rag = 0
    parse_fail_no_rag = 0
    parse_fail_with_rag = 0

    for example in tqdm(subset, total=limit):
        question = example["question"]
        gold = example["answer"]

        raw_no_rag = qwen_llm(question, rag=False)
        pred_no_rag, parsed_no_rag = new_parse_answer(raw_no_rag)
        if not parsed_no_rag:
            parse_fail_no_rag += 1
        if pred_no_rag == gold:
            correct_no_rag += 1

        entity_url_dict = extract_entities_and_urls(question)
        urls = list(entity_url_dict.values())
        docs = collect_docs(urls)
        retriever = create_retriever(docs)
        _, raw_with_rag = rag_chain(question, retriever)
        pred_with_rag, parsed_with_rag = new_parse_answer(raw_with_rag)
        if not parsed_with_rag:
            parse_fail_with_rag += 1
        if pred_with_rag == gold:
            correct_with_rag += 1

    acc_no_rag = correct_no_rag / limit
    acc_with_rag = correct_with_rag / limit

    return {
        "samples": limit,
        "acc_no_rag": acc_no_rag,
        "acc_with_rag": acc_with_rag,
        "parse_fail_no_rag": parse_fail_no_rag,
        "parse_fail_with_rag": parse_fail_with_rag,
    }


metrics_3_2 = reevaluate(val, limit=50)


![alt text](0c31d32cd7a1d7c2e010113398aaf5c9.png)

### Section 3.2: Analysis of RAG Enhancement with Reasoning

#### What We Did
Building on Section 3.1, we kept the entire RAG pipeline unchanged  
(i.e., NER-based entity extraction, Wikipedia retrieval, text chunking,  
SentenceTransformer embeddings, and Chroma-based retrieval).  
In this experiment, we introduced only two modifications:

1. **Prompt Change**  
   We required the model to generate explicit chain-of-thought reasoning  
   before selecting an answer, using the following format:
      Reasoning: \<short explanation\>
      Answer: \<A) or B) or C) or D)\>

2. **New Answer Parser**  
Because the model now produces reasoning in addition to the final answer,  
we implemented a new parsing function (`new_parse_answer`) to correctly  
extract the answer and track parsing failures.

We then re-evaluated the same two settings (WITHOUT RAG vs WITH RAG)  
on the first 50 samples using the Qwen3-4B-Instruct model with 4-bit  
quantization.

#### What the Results Mean

**Unexpected Decline**: Accuracy dropped from 48% (WITHOUT RAG) to 44% (WITH RAG),  
showing a **negative impact of RAG** when reasoning is required. This contrasts  
with Section 3.1, where simpler prompts showed marginal RAG benefits.

**Parser Degradation**: Parse failures increased from 5 (WITHOUT RAG) to 7 (WITH RAG).  
The combination of retrieved context and reasoning requirements appears to strain  
the 4-bit quantized model, causing more formatting errors.

**Key Insight**: Requiring explicit reasoning makes the model **more vulnerable to noise**  
from retrieved documents. The model must simultaneously:
1. Process long retrieved Wikipedia text that may contain irrelevant information  
2. Generate coherent medical reasoning  
3. Maintain the expected structured format (`Reasoning: ... / Answer: ...`)

This **dual burden** (long noisy context + structured reasoning) likely explains  
why RAG hurts performance in this setting. The retrieved content may distract the  
model or conflict with its internal medical knowledge.

**Possible Explanations**:
- **Low-quality retrieval**: NER-based retrieval may surface tangentially relevant Wikipedia pages.  
- **Context overflow**: The 4-bit quantized model has limited capacity to process long contexts.  
- **Reasoning interference**: External text may confuse or contradict the model’s reasoning.

**Comparison**: Section 3.1 used simpler prompts with no reasoning, while Section 3.3  
will introduce constrained decoding (Pydantic + Outlines) to enforce structured  
outputs—approaches that may better handle RAG than free-form reasoning.


## 3.3 Enhancement Experiments + Constrained Decoding

To address the parsing issues that you might have encountered in the previous experiments, you can use **Constrained Decoding**. Tools such as **Outlines** ([documentation](https://dottxt-ai.github.io/outlines/latest/features/core/output_types/#overview)) or **Litelines** ([GitHub](https://github.com/alonsosilvaallende/litelines)) can help enforce structured outputs from the model.

By using constrained decoding, the model can be guided to produce outputs in a consistent and parseable format, which should reduce the number of failed parses. However, constrained decoding may also change the model’s behavior. What would you do to allow the model to think freely while still ensuring that it produces a predictable and parseable answer?

Additionally, make sure to **track the number of cases that failed to parse**, as this metric will help assess the effectiveness of the constrained decoding strategy.

You might also want to add something "reasoning" to the Pydantic schema to allow the model to think a bit, which might leads to improve the model performance. Evaluate the first (at least) 50 samples of the dataset, comparing **ResponseWithoutReasoning** and **ResponseWithReasoning** to measure the effect of reasoning on performance (which the same prompt).

**Expected Result:**  
`ResponseWithReasoning` should outperform `ResponseWithoutReasoning` by a noticeable margin (4-6%)


**Instructions:**  
When reporting your results, **explain clearly**:

1. **What you did** – describe your setup, the models used, and the evaluation procedure.  
2. **What the results mean** – interpret the accuracy numbers and any observations.  

In [ ]:
!pip install outlines litelines -q

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field

class ResponseWithoutReasoning(BaseModel):
    answer: Literal["A)", "B)", "C)", "D)"]

class ResponseWithReasoning(BaseModel):
    reasoning: str
    answer: Literal["A)", "B)", "C)", "D)"]

In [ ]:
# Use this function to extract llm response

# Optional constrained-decoding backend (Outlines)
OUTLINES_AVAILABLE = False
_outlines_model = None
_outlines_generators = {}

try:
    from outlines import models as outlines_models
    from outlines import generate as outlines_generate
    OUTLINES_AVAILABLE = hasattr(outlines_generate, 'json')
except Exception as exc:
    outlines_models = None
    outlines_generate = None
    outlines_import_error = str(exc)
    OUTLINES_AVAILABLE = False

def get_outlines_generator(schema):
    """Return an Outlines JSON generator for the given schema if possible."""
    global _outlines_model
    if not OUTLINES_AVAILABLE:
        return None

    schema_name = schema.__name__
    if schema_name in _outlines_generators:
        return _outlines_generators[schema_name]

    try:
        if _outlines_model is None:
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
            _outlines_model = outlines_models.transformers(llm_name, device=device)
        generator = outlines_generate.json(_outlines_model, schema)
    except Exception as exc:
        print(f"Outlines generation unavailable, falling back to free decoding: {exc}")
        return None

    _outlines_generators[schema_name] = generator
    return generator


In [ ]:
# Use this function to extract llm response

def extract_response_from_json(text, schema=None):
    """
    Extract the model answer from a JSON object embedded in `text`.
    Returns:
        Tuple[str | None, bool]: (predicted answer, parsing success flag)
    """
    if not isinstance(text, str):
        return None, False

    match = re.search(r"{.*}", text, re.DOTALL)
    if not match:
        return None, False

    candidate = match.group(0)
    payload = None
    for snippet in (candidate, candidate.replace("'", '"')):
        try:
            payload = json.loads(snippet)
            break
        except json.JSONDecodeError:
            payload = None

    if payload is None:
        return None, False

    if schema is not None:
        try:
            validated = schema.model_validate(payload)
        except ValidationError:
            return None, False
        return validated.answer, True

    answer = payload.get("answer") if isinstance(payload, dict) else None
    return (answer, True) if isinstance(answer, str) else (None, False)


In [ ]:
STRUCTURED_SYSTEM_PROMPT = (
    "You are an expert medical student. Use any provided context to answer the "
    "multiple-choice question. Always respond with a single valid JSON object "
    "that uses double quotes and matches the requested schema."
)

def call_structured_llm(user_prompt):
    messages = [
        {"role": "system", "content": STRUCTURED_SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True
    ).to(llm.device)

    outputs = llm.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        temperature=0.0
    )
    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:-1])


def build_structured_prompt(question, context, schema=None, include_schema=False):
    context_block = context.strip() if context else 'No additional context found.'
    prompt = (
        "Answer the question below using evidence from the context when possible.\n"
        f"Question:\n{question}\n\nContext:\n{context_block}\n\n"
        "Return a single JSON object with your final answer.\n"
    )
    full_prompt = ''.join(prompt)

    if include_schema and schema is not None:
        schema_json = json.dumps(schema.model_json_schema(), indent=2)
        full_prompt += f"Schema to follow:\n{schema_json}\n\n"

    full_prompt += 'Only output JSON. Do not add explanations or markdown fences.'
    return full_prompt


def build_context_for_question(question):
    entity_url_dict = extract_entities_and_urls(question)
    urls = list(entity_url_dict.values())
    docs = collect_docs(urls)
    retriever = create_retriever(docs)
    if retriever is None:
        return ''

    try:
        retrieved_docs = retriever.get_relevant_documents(question)
    except Exception:
        retrieved_docs = []

    context = '\n\n'.join(
        doc.page_content.strip()
        for doc in retrieved_docs
        if getattr(doc, 'page_content', None)
    ).strip()
    return context


def structured_answer(question, context, schema):
    generator = get_outlines_generator(schema)
    prompt = build_structured_prompt(
        question,
        context,
        schema=schema,
        include_schema=not OUTLINES_AVAILABLE
    )

    if generator is not None:
        try:
            structured = generator(prompt)
            if hasattr(structured, 'answer'):
                return structured.answer, True
            if isinstance(structured, dict) and isinstance(structured.get('answer'), str):
                return structured['answer'], True
        except Exception as exc:
            print(f"Outlines generation failed, fallback to free decoding: {exc}")

    raw = call_structured_llm(prompt)
    return extract_response_from_json(raw, schema)


In [ ]:
def constrained_decoding_experiment(val, limit=50):
    """Compare constrained decoding with and without reasoning."""
    from tqdm import tqdm

    dataset_size = len(val)
    if dataset_size == 0:
        raise ValueError('Dataset is empty.')

    limit = min(limit, dataset_size)
    subset = val.select(range(limit)) if hasattr(val, 'select') else val[:limit]

    stats = {
        'without_reasoning': {'correct': 0, 'parse_fail': 0},
        'with_reasoning': {'correct': 0, 'parse_fail': 0}
    }

    for example in tqdm(subset, total=limit):
        question = example['question']
        gold = example['answer']
        context = build_context_for_question(question)

        for label, schema in [
            ('without_reasoning', ResponseWithoutReasoning),
            ('with_reasoning', ResponseWithReasoning),
        ]:
            pred, parsed = structured_answer(question, context, schema)
            if not parsed:
                stats[label]['parse_fail'] += 1
            if pred == gold:
                stats[label]['correct'] += 1

    for label in stats:
        stats[label]['samples'] = limit
        stats[label]['accuracy'] = stats[label]['correct'] / limit

    return stats


metrics_3_3 = constrained_decoding_experiment(val, limit=50)


![alt text](e3cef498275829ec6f8f9ec88192b85e.png)

### Section 3.3: Analysis of Constrained Decoding with/without Reasoning

#### What We Did

In this experiment, we use **constrained decoding** via the Outlines library to enforce strict adherence to Pydantic JSON schemas during generation. 
Both conditions use the same RAG pipeline (NER entity extraction, Wikipedia retrieval, Chroma vector store with nomic-embed-text-v1 embeddings) and the same base prompt. The key difference is the enforcement of JSON schema through Outlines' constrained decoding, which guides token generation to produce valid JSON outputs that match the specified Pydantic models.

We tracked both accuracy and parsing failures for each schema type.

#### What the Results Mean

**Reasoning Improves Accuracy**: Accuracy increased from 42% (without reasoning) to 50% (with reasoning), an **8-percentage-point improvement**. This demonstrates that requiring the model to articulate its reasoning process leads to better medical question answering—the model "thinks through" the problem more carefully before committing to an answer.

**Constrained Decoding Reduces Parse Failures**: Parse failures remain very low (3 for without reasoning, 4 for with reasoning), confirming that constrained decoding effectively enforces format compliance. Even when generating longer reasoning text, the model maintains proper JSON structure thanks to the Outlines library's token-level constraints.

**Key Contrast with Section 3.2**: In Section 3.2 (without constrained decoding), adding reasoning + RAG **decreased** accuracy (48% → 44%) and **increased** parse failures (5-7 failures). Here, constrained decoding solves the formatting problem, allowing reasoning to provide its intended benefit. This suggests that the previous decline was partly due to the model struggling with free-form text generation under cognitive load—it had to simultaneously manage content quality AND output format.

**Why Constrained Decoding Helps**:
- **Reduced cognitive burden**: The model no longer needs to "worry" about JSON formatting—the Outlines library handles this at the token generation level, letting the model focus purely on content.
- **Guaranteed parseability**: Unlike free-form generation where the model might produce malformed JSON, constrained decoding ensures every output is valid.
- **Better reasoning utilization**: With format concerns removed, the model can fully leverage its reasoning capabilities to analyze medical questions more deeply.

**Limitation**: While constrained decoding dramatically improves reliability, parse failures aren't completely eliminated (3-4 instances). This could be due to:
- Tokenization mismatches between the model's vocabulary and JSON requirements
- Edge cases where the model attempts to generate text fundamentally incompatible with the schema's constraints
- Retrieval failures or empty contexts causing generation issues

**Overall**: Constrained decoding + reasoning is the **most effective combination**, achieving 50% accuracy with minimal format errors. This validates the approach of using structured generation libraries (like Outlines or Litelines) for reliable LLM outputs in medical QA tasks. The results show that RAG can be beneficial when combined with proper output structure enforcement, addressing both the content quality (through reasoning) and reliability (through constrained decoding) challenges observed in earlier experiments.